# Importing libraries & data


In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [3]:
!gdown 1b_FZ_6ALQsD9ZaOFuLN4n0IXXwwemlF6

Downloading...
From: https://drive.google.com/uc?id=1b_FZ_6ALQsD9ZaOFuLN4n0IXXwwemlF6
To: /content/Cab_Data.csv
100% 21.2M/21.2M [00:00<00:00, 155MB/s]


In [4]:
!gdown 1clgqENGZgOCakoErCuPA8gQMpGplTuNH

Downloading...
From: https://drive.google.com/uc?id=1clgqENGZgOCakoErCuPA8gQMpGplTuNH
To: /content/City.csv
100% 759/759 [00:00<00:00, 3.06MB/s]


In [5]:
!gdown 15HYCv83IYNtq_Nyql-GbrF7KkzIXwwZM

Downloading...
From: https://drive.google.com/uc?id=15HYCv83IYNtq_Nyql-GbrF7KkzIXwwZM
To: /content/Customer_ID.csv
100% 1.05M/1.05M [00:00<00:00, 75.8MB/s]


In [6]:
!gdown 1b_I-22au9PREgXaQho2d0TLRYoB6uzWV

Downloading...
From: https://drive.google.com/uc?id=1b_I-22au9PREgXaQho2d0TLRYoB6uzWV
To: /content/Transaction_ID.csv
100% 9.00M/9.00M [00:00<00:00, 79.5MB/s]


In [77]:
cab_data = pd.read_csv("/content/Cab_Data.csv")
city = pd.read_csv("/content/City.csv")
customer_id = pd.read_csv("/content/Customer_ID.csv")
transaction_id = pd.read_csv("/content/Transaction_ID.csv")

# Data Preprocessing

In [78]:
cab_data.head()

,Transaction ID,Date of Travel,Company,City,KM Travelled,Price Charged,Cost of Trip
0,10000011,42377,Pink Cab,ATLANTA GA,30.45,370.95,313.635
1,10000012,42375,Pink Cab,ATLANTA GA,28.62,358.52,334.854
2,10000013,42371,Pink Cab,ATLANTA GA,9.04,125.20,97.632
3,10000014,42376,Pink Cab,ATLANTA GA,33.17,377.40,351.602
4,10000015,42372,Pink Cab,ATLANTA GA,8.73,114.62,97.776


In [79]:
cab_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359392 entries, 0 to 359391
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Transaction ID  359392 non-null  int64  
 1   Date of Travel  359392 non-null  int64  
 2   Company         359392 non-null  object 
 3   City            359392 non-null  object 
 4   KM Travelled    359392 non-null  float64
 5   Price Charged   359392 non-null  float64
 6   Cost of Trip    359392 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 19.2+ MB


---
* Here travel date is not in proper format so we have to convert it into proper format also it is in object dtype
---

In [80]:
def date_travel(d):
    start = datetime.datetime(1900, 1, 1)
    return start + datetime.timedelta(d)

In [81]:
cab_data["Date of Travel"] = cab_data["Date of Travel"].apply(date_travel)

In [82]:
cab_data.head()

,Transaction ID,Date of Travel,Company,City,KM Travelled,Price Charged,Cost of Trip
0,10000011,2016-01-10,Pink Cab,ATLANTA GA,30.45,370.95,313.635
1,10000012,2016-01-08,Pink Cab,ATLANTA GA,28.62,358.52,334.854
2,10000013,2016-01-04,Pink Cab,ATLANTA GA,9.04,125.20,97.632
3,10000014,2016-01-09,Pink Cab,ATLANTA GA,33.17,377.40,351.602
4,10000015,2016-01-05,Pink Cab,ATLANTA GA,8.73,114.62,97.776


In [83]:
cab_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359392 entries, 0 to 359391
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Transaction ID  359392 non-null  int64         
 1   Date of Travel  359392 non-null  datetime64[ns]
 2   Company         359392 non-null  object        
 3   City            359392 non-null  object        
 4   KM Travelled    359392 non-null  float64       
 5   Price Charged   359392 non-null  float64       
 6   Cost of Trip    359392 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 19.2+ MB


In [87]:
cab_data.describe()

,Transaction ID,Date of Travel,KM Travelled,Price Charged,Cost of Trip
count,3.593920e+05,359392,359392.000000,359392.000000,359392.000000
mean,1.022076e+07,2017-08-19 01:37:55.042293760,22.567254,423.443311,286.190113
min,1.000001e+07,2016-01-04 00:00:00,1.900000,15.600000,19.000000
25%,1.011081e+07,2016-11-25 00:00:00,12.000000,206.437500,151.200000
50%,1.022104e+07,2017-09-12 00:00:00,22.440000,386.360000,282.480000
75%,1.033094e+07,2018-05-14 00:00:00,32.960000,583.660000,413.683200
max,1.044011e+07,2019-01-02 00:00:00,48.000000,2048.030000,691.200000
std,1.268058e+05,NaN,12.233526,274.378911,157.993661


In [84]:
cab_data.isnull().sum()

,0
Transaction ID,0
Date of Travel,0
Company,0
City,0
KM Travelled,0
Price Charged,0
Cost of Trip,0


In [109]:
cab_data.duplicated().sum()

0

In [85]:
cab_data["Company"].value_counts()

,count
Company,
Yellow Cab,274681
Pink Cab,84711


In [86]:
cab_data["City"].value_counts()

,count
City,
NEW YORK NY,99885
CHICAGO IL,56625
LOS ANGELES CA,48033
WASHINGTON DC,43737
BOSTON MA,29692
SAN DIEGO CA,20488
SILICON VALLEY,8519
SEATTLE WA,7997
ATLANTA GA,7557


In [ ]:
cab_data["Company"].value_counts()

In [89]:
city.head()

,City,Population,Users
0,NEW YORK NY,"8,405,837","302,149"
1,CHICAGO IL,"1,955,130","164,468"
2,LOS ANGELES CA,"1,595,037","144,132"
3,MIAMI FL,"1,339,155","17,675"
4,SILICON VALLEY,"1,177,609","27,247"


In [88]:
city.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   City        20 non-null     object
 1   Population  20 non-null     object
 2   Users       20 non-null     object
dtypes: object(3)
memory usage: 608.0+ bytes


---
* Here Population and Users are showing object data type but they are integer data type.
---

In [92]:
city["Population"] = city["Population"].str.replace(",","").astype(int)
city["Users"] = city["Users"].str.replace(",","").astype(int)

In [93]:
city.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   City        20 non-null     object
 1   Population  20 non-null     int64 
 2   Users       20 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 608.0+ bytes


In [94]:
city.describe()

,Population,Users
count,2.000000e+01,20.000000
mean,1.231592e+06,64520.650000
std,1.740127e+06,83499.375289
min,2.489680e+05,3643.000000
25%,6.086372e+05,11633.250000
50%,7.845590e+05,23429.000000
75%,1.067041e+06,91766.000000
max,8.405837e+06,302149.000000


In [111]:
city.isnull().sum()

,0
City,0
Population,0
Users,0


In [112]:
city.duplicated().sum()

0

In [95]:
customer_id.head()

,Customer ID,Gender,Age,Income (USD/Month)
0,29290,Male,28,10813
1,27703,Male,27,9237
2,28712,Male,53,11242
3,28020,Male,23,23327
4,27182,Male,33,8536


In [96]:
customer_id.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49171 entries, 0 to 49170
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Customer ID         49171 non-null  int64 
 1   Gender              49171 non-null  object
 2   Age                 49171 non-null  int64 
 3   Income (USD/Month)  49171 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


In [97]:
customer_id.describe()

,Customer ID,Age,Income (USD/Month)
count,49171.000000,49171.000000,49171.000000
mean,28398.252283,35.363121,15015.631856
std,17714.137333,12.599066,8002.208253
min,1.000000,18.000000,2000.000000
25%,12654.500000,25.000000,8289.500000
50%,27631.000000,33.000000,14656.000000
75%,43284.500000,42.000000,21035.000000
max,60000.000000,65.000000,35000.000000


In [113]:
customer_id.isnull().sum()

,0
Customer ID,0
Gender,0
Age,0
Income (USD/Month),0


In [115]:
customer_id.duplicated().sum()

0

In [98]:
customer_id["Gender"].value_counts()

,count
Gender,
Male,26562
Female,22609


In [99]:
transaction_id.head()

,Transaction ID,Customer ID,Payment_Mode
0,10000011,29290,Card
1,10000012,27703,Card
2,10000013,28712,Cash
3,10000014,28020,Cash
4,10000015,27182,Card


In [100]:
transaction_id.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440098 entries, 0 to 440097
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Transaction ID  440098 non-null  int64 
 1   Customer ID     440098 non-null  int64 
 2   Payment_Mode    440098 non-null  object
dtypes: int64(2), object(1)
memory usage: 10.1+ MB


In [101]:
transaction_id.describe()

,Transaction ID,Customer ID
count,4.400980e+05,440098.000000
mean,1.022006e+07,23619.513120
std,1.270455e+05,21195.549816
min,1.000001e+07,1.000000
25%,1.011004e+07,3530.000000
50%,1.022006e+07,15168.000000
75%,1.033008e+07,43884.000000
max,1.044011e+07,60000.000000


In [116]:
transaction_id.isnull().sum()

,0
Transaction ID,0
Customer ID,0
Payment_Mode,0


In [117]:
transaction_id.duplicated().sum()

0

In [102]:
transaction_id["Payment_Mode"].value_counts()

,count
Payment_Mode,
Card,263991
Cash,176107


---
* Merging all Data
---

In [105]:
df = cab_data.merge(city, on="City").merge(transaction_id, on="Transaction ID")

In [106]:
df = df.merge(customer_id, on="Customer ID")
df

,Transaction ID,Date of Travel,Company,City,KM Travelled,Price Charged,Cost of Trip,Population,Users,Customer ID,Payment_Mode,Gender,Age,Income (USD/Month)
0,10000011,2016-01-10,Pink Cab,ATLANTA GA,30.45,370.95,313.6350,814885,24701,29290,Card,Male,28,10813
1,10000012,2016-01-08,Pink Cab,ATLANTA GA,28.62,358.52,334.8540,814885,24701,27703,Card,Male,27,9237
2,10000013,2016-01-04,Pink Cab,ATLANTA GA,9.04,125.20,97.6320,814885,24701,28712,Cash,Male,53,11242
3,10000014,2016-01-09,Pink Cab,ATLANTA GA,33.17,377.40,351.6020,814885,24701,28020,Cash,Male,23,23327
4,10000015,2016-01-05,Pink Cab,ATLANTA GA,8.73,114.62,97.7760,814885,24701,27182,Card,Male,33,8536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359387,10440101,2018-01-10,Yellow Cab,WASHINGTON DC,4.80,69.24,63.3600,418859,127001,52392,Cash,Male,24,15651
359388,10440104,2018-01-06,Yellow Cab,WASHINGTON DC,8.40,113.75,106.8480,418859,127001,53286,Cash,Male,32,6528
359389,10440105,2018-01-07,Yellow Cab,WASHINGTON DC,27.75,437.07,349.6500,418859,127001,52265,Cash,Male,56,7966
359390,10440106,2018-01-07,Yellow Cab,WASHINGTON DC,8.80,146.19,114.0480,418859,127001,52175,Card,Male,32,6423


In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359392 entries, 0 to 359391
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   Transaction ID      359392 non-null  int64         
 1   Date of Travel      359392 non-null  datetime64[ns]
 2   Company             359392 non-null  object        
 3   City                359392 non-null  object        
 4   KM Travelled        359392 non-null  float64       
 5   Price Charged       359392 non-null  float64       
 6   Cost of Trip        359392 non-null  float64       
 7   Population          359392 non-null  int64         
 8   Users               359392 non-null  int64         
 9   Customer ID         359392 non-null  int64         
 10  Payment_Mode        359392 non-null  object        
 11  Gender              359392 non-null  object        
 12  Age                 359392 non-null  int64         
 13  Income (USD/Month)  359392 no

# EDA